In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2, norm, mannwhitneyu, ks_2samp
from scipy import stats

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import statsmodels.formula.api as smf
from plotly.subplots import make_subplots
from statsmodels.formula.api import logit
from scipy.stats import chi2_contingency

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Para ignorar warnings
import warnings
warnings.filterwarnings('ignore')

df = (
    pd.read_csv("stroke.csv")
            .drop("id", axis=1)
)

# Prueba MCAR de Jamshidial Jalal

In [3]:

def test_mcar_jamshidian_jalal(df):
    """
    Implementa la prueba MCAR de Jamshidian y Jalal de forma aproximada.
    
    Parámetros:
    - df: DataFrame con valores numéricos y datos faltantes.
    
    Retorna:
    - p-valor de la prueba (si p > 0.05, los datos son MCAR).
    """
    # Identificar valores nulos
    missing_mask = df.isnull()
    missing_patterns = missing_mask.astype(int).groupby(list(missing_mask.columns)).size()

    # Construcción de matriz de medias y covarianzas
    complete_cases = df.dropna()
    mean_complete = complete_cases.mean()
    cov_complete = complete_cases.cov()

    # Calcular estadístico de prueba (basado en comparación de patrones)
    test_stat = 0
    for pattern, count in missing_patterns.items():
        missing_indices = np.where(pattern)[0]
        observed = df.iloc[:, missing_indices].dropna()
        mean_observed = observed.mean()
        cov_observed = observed.cov()

        # Diferencia de medias
        diff = mean_observed - mean_complete[missing_indices]

        # Calcular estadístico chi-cuadrado
        try:
            inv_cov = np.linalg.inv(cov_observed)
            test_stat += count * (diff.T @ inv_cov @ diff)
        except np.linalg.LinAlgError:
            continue  # Saltar casos donde la matriz de covarianza no es invertible

    # Grados de libertad y p-valor
    df_degrees = sum(missing_patterns) - len(df.columns)
    p_value = 1 - chi2.cdf(test_stat, df_degrees)

    return test_stat, p_value

# 📌 Cargar dataset
df = pd.read_csv("stroke.csv")

# 📌 Seleccionar solo variables numéricas
df_numeric = df[['age', 'avg_glucose_level', 'bmi']]

# 📌 Aplicar prueba MCAR de Jamshidian y Jalal
test_stat, p_value = test_mcar_jamshidian_jalal(df_numeric)

# 📌 Mostrar resultado
print(f"Estadístico de prueba: {test_stat:.4f}, p-valor: {p_value:.4f}")

# 📌 Interpretación:
if p_value > 0.05:
    print("✅ No se rechaza H0: Los datos faltantes pueden ser MCAR.")
else:
    print("❌ Se rechaza H0: Los datos faltantes NO son completamente al azar (posiblemente MAR o MNAR).")

Estadístico de prueba: 0.0000, p-valor: 1.0000
✅ No se rechaza H0: Los datos faltantes pueden ser MCAR.


***
# Pruebas MAR

## Kolmogorov-Smirnov:

In [13]:
df['bmi_missing'] = df['bmi'].isna().astype(int)

In [15]:
# Subconjunto donde BMI está ausente
age_missing = df.loc[df['bmi_missing'] == 1, 'age'].dropna()   #luego cambiar a avg_glucose_level

# Subconjunto donde BMI está presente
age_not_missing = df.loc[df['bmi_missing'] == 0, 'age'].dropna() #Luego cambiar a avg_glucose_level

In [21]:

# 📌 Separar los datos completos e incompletos de BMI
df_complete = df_numeric.dropna()
df_missing = df_numeric[df_numeric['bmi'].isnull()]

# 📌 Prueba de Kolmogorov-Smirnov
ks_stat, ks_p_value = ks_2samp(age_missing, age_not_missing)
print("KS Statistic:", ks_stat)
print("p-value:", ks_p_value)

KS Statistic: 0.21779065560362781
p-value: 1.6067879142273805e-08


## U de Mann-Whitney:

In [70]:
df['bmi_missing'] = df['bmi'].isna().astype(int)

In [58]:
# Subconjunto donde BMI está ausente
age_missing = df.loc[df['bmi_missing'] == 1, 'age'].dropna()   #luego cambiar a avg_glucose_level

# Subconjunto donde BMI está presente
age_not_missing = df.loc[df['bmi_missing'] == 0, 'age'].dropna() #Luego cambiar a avg_glucose_level


In [60]:
u_stat, p_value_u = stats.mannwhitneyu(age_missing, age_not_missing, alternative='two-sided')

print("U-Statistic:", u_stat)
print("p-value:", p_value_u)

U-Statistic: 610499.0
p-value: 1.098407129730987e-08


Los resultados mostrados, indican que existen asociaciones significativas entre los
valores nulos y variables como la edad y el género. Estos resultados sugieren que los datos faltantes
no son completamente aleatorios y probablemente siguen un patrón MAR. 